In [1]:
# def playMode(lamda, alpha, beta):
    
# lamda = 7635274
# alpha = 0.000000625
# beta = 0.000008

In [2]:
import pyaudio
import numpy as np
import math

#### Basic Parameters
$$
\omega = \sqrt \lambda \\
\xi = \frac{\alpha + \beta \lambda}{2\omega} \\
\omega_d = \omega\sqrt{1- \xi^2}
$$

In [3]:
lamda = 7635274.24
alpha = 0.000000625
beta = 0.000008

fs = 44100       # sampling rate, Hz, must be integer
duration = 1.0   # in seconds, may be float

omega = math.sqrt(lamda)
# freq = omega / 2 / np.pi
ksi = (alpha + beta * lamda) / 2 / omega
omega_d = omega * math.sqrt(1 - ksi * ksi)


In [4]:
print(omega)
print(omega_d)

2763.2
2763.031212523194


#### Construct Samples
$$
q(t) = \underbrace{\frac 1 {\omega_d} e^{-\xi \omega t}}_\text{scale or amplitude} \sin (\omega_d t)
$$

In [5]:
time = np.arange(fs * duration) / fs
scale = np.exp(time * (-1) * ksi * omega)
# scale = 1 / omega_d * np.exp(time * (-1) * ksi * omega)   # 单个模态也乘上1/omgega_d会让声音特别小

In [6]:
print(scale[:5000])

[1.         0.9993077  0.99861588 ... 0.03140991 0.03138817 0.03136644]


In [7]:
# generate samples, note conversion to float32 array
samples = (np.sin( omega_d * time) * scale).astype(np.float32)

In [8]:
samples

array([ 0.0000000e+00,  6.2569439e-02,  1.2480688e-01, ...,
       -5.3628305e-14, -5.4122617e-14, -5.4403974e-14], dtype=float32)

#### Play

In [9]:
# original samples
# samples = (np.sin( 2 * np.pi * freq * np.arange(fs * duration) / fs)).astype(np.float32)

# for paFloat32 sample values must be in range [-1.0, 1.0]
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=fs,
                output=True)

# play
stream.write(samples.tobytes())

stream.stop_stream()
stream.close()

p.terminate()

#### Save to wav

In [ ]:
# save to wav file
from scipy.io.wavfile import write
write('./test.wav', fs, samples)